In [1]:
from infrastructure.database.mongo_client import MongoDBClient
from infrastructure.repository import YouTubeContentRepository
import os
from dotenv import load_dotenv

load_dotenv()

# MongoDB 클라이언트 초기화
client = MongoDBClient(uri=os.environ['MONGO_CONNECTION_STRING'])
client.connect()

# 저장소 초기화
repository = YouTubeContentRepository(client)

contents = repository.find_all()
content = contents[0]
print(content.title)

Connected to MongoDB
2D 오브젝트🧍만들기 [유니티 뱀서라이크 01]


In [2]:
script_whisper = content.script_whisper
script_auto = content.script_auto

print(script_whisper)
print("----"*7)
print(script_auto)

YouTubeScript(script=' 자 여러분들 반갑습니다 골드메탈입니다 이번엔 뱀스라이크 언더티 서바이버 개발의 첫 번째 ...', chunks=[(0.0-11.24:  자 여러분들 반갑습니다 골...), (11.24-15.56:  이번엔 뱀스라이크 언더티 ...), (15.56-24.67:  유니티를 한번 설치해 보고...), ...] (427 chunks))
----------------------------
YouTubeScript(script='자 여러분들 반갑습니다 이번에는 뱀사이크 언제든 서바이버 개발에 첫 번째 시간으로서 일단 ...', chunks=[(7.799-11.88: 자 여러분들 반갑습니다 이번...), (12.059-14.34: 뱀사이크...), (13.019-16.379: 언제든 서바이버 개발에 첫 ...), ...] (634 chunks))


In [3]:
from pydantic import BaseModel, Field
from typing import List, Tuple
from langchain.output_parsers import PydanticOutputParser

# YouTubeScriptChunk의 Pydantic 모델 정의
class YouTubeScriptChunkModel(BaseModel):
    timestamp: Tuple[int, int] = Field(..., description="Chunk의 시작 및 끝 타임스탬프")
    text: str = Field(..., description="해당 시간의 스크립트 텍스트")

class YouTubeScriptChunksModel(BaseModel):
    chunks: List[YouTubeScriptChunkModel] = Field(..., description="타임스탬프별 텍스트 조각 리스트")

pydantic_script_refine_output_parser = PydanticOutputParser(pydantic_object=YouTubeScriptChunksModel)

In [4]:
from langchain.prompts import PromptTemplate

script_refine_prompt_template = PromptTemplate(
    partial_variables={'output_format': pydantic_script_refine_output_parser.get_format_instructions()},
    input_variables=["description", "script_chunk"],
    template="""
    다음은 유튜브 영상의 설명과 자막입니다.
    유튜브 영상 설명:
    {description}

    - 자동 생성 자막: {script_chunk}

    위 정보를 참고하여 더 정확하고 매끄러운 자막 청크를 생성해주세요.
    각 청크는 타임스탬프와 텍스트로 구성되어야 합니다.

    - 오타가 났거나 애매한 용어는 유튜브 영상 설명 내용을 참고해주세요.
    - 타임스탬프를 엄격하게 지켜주세요.

    아래 JSON 형식에 맞춰 결과를 반환하세요:
    {output_format}
    """
)

In [5]:
from langchain_openai import ChatOpenAI

openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ['OPENAI_API_KEY'])

In [6]:
script_refine_chain = script_refine_prompt_template | openai_llm | pydantic_script_refine_output_parser

In [7]:
input_data = {
        "description": content.description,
        "script_chunk": "\n".join(
            [f"({int(chunk.start_time)}-{int(chunk.end_time)}): {chunk.text}" for chunk in content.script_auto.chunks]
        )
    }

print(input_data)

{'description': '뱀서라이크 유니티 개발 강좌 첫번째 시간!\n유니티를 처음 만지는 입문자 분들도 따라하기 쉽도록\n2D 오브젝트를 유니티에서 어떻게 만드는지 차근차근 설명해드리고 있습니다.\n본격적인 움직임 구현은 다음 시간에 다룰 예정입니다.  🤗\n\n📚 뱀서라이크 개발 채널\nhttps://www.youtube.com/playlist?list=PLO-mt5Iu5TeZF8xMHqtT_DhAPKmjF6i3x\n\n📦 언데드서바이버 에셋\nhttps://prf.hn/l/aQNdNmA\n\n📖 챕터\n01 00:00 프로젝트 시작하기\n02 07:00 에셋 가져오기\n03 09:24 오브젝트 만들기\n04 17:11 컴포넌트 추가하기\n\n#유니티강좌 #유니티기초 #뱀서라이크 #뱀파이어서바이버', 'script_chunk': '(7-11): 자 여러분들 반갑습니다 이번에는\n(12-14): 뱀사이크\n(13-16): 언제든 서바이버 개발에 첫 번째\n(14-19): 시간으로서 일단 유니티를 한번\n(16-20): 설치해보고 자 브레이트 시합과 그\n(19-22): 다음에 플레이어를\n(20-23): 간단하게 만들어 보는\n(22-25): 방법에 대해서 같이 한번\n(23-27): 진행 좀 해보도록 하겠습니다 그래서\n(25-30): 여러분들 뭐 네이버나 아니면\n(27-31): 구글에다가 유니티 검색하셔서 이렇게\n(30-32): 공식 홈페이지\n(31-36): 들어오시면 되는데요\n(32-38): 유니티.com/h 다운로드\n(36-40): 버튼이 있어요 자 들어가시고요\n(40-43): 그러면 여기 또 윈도우용 다운로드\n(43-45): 했습니다\n(43-47): 물론 또 이제 ius는 다운로드도\n(45-50): 아래에서 할 수 있으니깐요 확인을\n(47-53): 하시면 좋겠습니다 그래서 버튼 눌러서\n(50-53): 자 다운로드 하시고요\n(54-58): 그러면 유니티 에디터 설치 파일이\n(57-60): 나오는게 아니고\n(58-62): 

In [8]:
import tiktoken

encoder = tiktoken.encoding_for_model("gpt-4o-mini")

prompt = script_refine_prompt_template.format(**input_data)
print(prompt)
tokens = len(encoder.encode(prompt))
print(tokens)


    다음은 유튜브 영상의 설명과 자막입니다.
    유튜브 영상 설명:
    뱀서라이크 유니티 개발 강좌 첫번째 시간!
유니티를 처음 만지는 입문자 분들도 따라하기 쉽도록
2D 오브젝트를 유니티에서 어떻게 만드는지 차근차근 설명해드리고 있습니다.
본격적인 움직임 구현은 다음 시간에 다룰 예정입니다.  🤗

📚 뱀서라이크 개발 채널
https://www.youtube.com/playlist?list=PLO-mt5Iu5TeZF8xMHqtT_DhAPKmjF6i3x

📦 언데드서바이버 에셋
https://prf.hn/l/aQNdNmA

📖 챕터
01 00:00 프로젝트 시작하기
02 07:00 에셋 가져오기
03 09:24 오브젝트 만들기
04 17:11 컴포넌트 추가하기

#유니티강좌 #유니티기초 #뱀서라이크 #뱀파이어서바이버

    - 자동 생성 자막: (7-11): 자 여러분들 반갑습니다 이번에는
(12-14): 뱀사이크
(13-16): 언제든 서바이버 개발에 첫 번째
(14-19): 시간으로서 일단 유니티를 한번
(16-20): 설치해보고 자 브레이트 시합과 그
(19-22): 다음에 플레이어를
(20-23): 간단하게 만들어 보는
(22-25): 방법에 대해서 같이 한번
(23-27): 진행 좀 해보도록 하겠습니다 그래서
(25-30): 여러분들 뭐 네이버나 아니면
(27-31): 구글에다가 유니티 검색하셔서 이렇게
(30-32): 공식 홈페이지
(31-36): 들어오시면 되는데요
(32-38): 유니티.com/h 다운로드
(36-40): 버튼이 있어요 자 들어가시고요
(40-43): 그러면 여기 또 윈도우용 다운로드
(43-45): 했습니다
(43-47): 물론 또 이제 ius는 다운로드도
(45-50): 아래에서 할 수 있으니깐요 확인을
(47-53): 하시면 좋겠습니다 그래서 버튼 눌러서
(50-53): 자 다운로드 하시고요
(54-58): 그러면 유니티 에디터 설치 파일이
(57-60): 나오는게 아니고
(58-62): 허브 설치 파일이 나옵

In [9]:
response = script_refine_chain.invoke(input=input_data)

In [10]:
print(response)

chunks=[YouTubeScriptChunkModel(timestamp=(7, 11), text='자 여러분들 반갑습니다. 이번에는 뱀서라이크 언데드 서바이버 개발의 첫 번째 시간으로서 유니티를 한번 설치해보고 플레이어를 간단하게 만들어 보는 방법에 대해서 같이 진행해 보도록 하겠습니다.'), YouTubeScriptChunkModel(timestamp=(12, 20), text='여러분들 뭐 네이버나 아니면 구글에다가 유니티 검색하셔서 공식 홈페이지에 들어오시면 됩니다.'), YouTubeScriptChunkModel(timestamp=(20, 26), text='유니티.com에 가셔서 다운로드 버튼을 눌러주시면 됩니다.'), YouTubeScriptChunkModel(timestamp=(26, 35), text='그러면 여기서 윈도우용 다운로드가 나오는데요, 거기서 다운로드하시면 좋겠습니다.'), YouTubeScriptChunkModel(timestamp=(35, 43), text='그래서 버튼을 눌러서 유니티 Hub 설치 파일을 다운로드하셔야 합니다.'), YouTubeScriptChunkModel(timestamp=(43, 54), text='유니티 에디터 설치 파일이 나오는 것이 아니라 허브 설치 파일이 나옵니다.'), YouTubeScriptChunkModel(timestamp=(54, 62), text='이걸 눌러서 유니티 허브부터 설치하시는 것이 좋겠습니다.'), YouTubeScriptChunkModel(timestamp=(62, 70), text='설치 동의함을 눌러주시고.'), YouTubeScriptChunkModel(timestamp=(70, 78), text='유니티 허브나 다른 것을 설치할 때 주 드라이브 내에 따로 유니티 허브 폴더를 만드시고 거기에 설치하는 것을 추천드립니다.'), YouTubeScriptChunkModel(timestamp=(78, 90), text='그래야 유니티 파일들을 관리하기가 쉽거든요.'), You

In [11]:
from domain import  YouTubeScript, YouTubeScriptChunk
chunks = [
        YouTubeScriptChunk(
            timestamp=chunk.timestamp,
            text=chunk.text,
        )
        for chunk in response.chunks
    ]
# 전체 스크립트를 청크 텍스트의 조합으로 생성
script = " ".join(chunk.text for chunk in chunks)

refine_script = YouTubeScript(script, chunks)
print(refine_script)

YouTubeScript(script='자 여러분들 반갑습니다. 이번에는 뱀서라이크 언데드 서바이버 개발의 첫 번째 시간으로서 유...', chunks=[(7-11: 자 여러분들 반갑습니다. 이...), (12-20: 여러분들 뭐 네이버나 아니면...), (20-26: 유니티.com에 가셔서 다운...), ...] (96 chunks))


In [12]:
#content.set_script(refine_script)
#repository.save(content)

In [13]:
print(script)

자 여러분들 반갑습니다. 이번에는 뱀서라이크 언데드 서바이버 개발의 첫 번째 시간으로서 유니티를 한번 설치해보고 플레이어를 간단하게 만들어 보는 방법에 대해서 같이 진행해 보도록 하겠습니다. 여러분들 뭐 네이버나 아니면 구글에다가 유니티 검색하셔서 공식 홈페이지에 들어오시면 됩니다. 유니티.com에 가셔서 다운로드 버튼을 눌러주시면 됩니다. 그러면 여기서 윈도우용 다운로드가 나오는데요, 거기서 다운로드하시면 좋겠습니다. 그래서 버튼을 눌러서 유니티 Hub 설치 파일을 다운로드하셔야 합니다. 유니티 에디터 설치 파일이 나오는 것이 아니라 허브 설치 파일이 나옵니다. 이걸 눌러서 유니티 허브부터 설치하시는 것이 좋겠습니다. 설치 동의함을 눌러주시고. 유니티 허브나 다른 것을 설치할 때 주 드라이브 내에 따로 유니티 허브 폴더를 만드시고 거기에 설치하는 것을 추천드립니다. 그래야 유니티 파일들을 관리하기가 쉽거든요. 그래서 이렇게 깔끔하게 정리해서 설치하는 것을 추천드립니다. 설치를 완료했으면 이렇게 유니티 허브가 나올 겁니다. 로그인 하셔야 하구요, 유니티 개인 계정이 필요합니다. 로그인 하시면 라이센스를 다 가지게 됩니다. 유니티는 보통 유니티 에디터입니다. 에디터는 여기 설치 탭에서 설치할 수 있습니다. 저는 이미 2021.2 버전과 2021 버전이 같이 설치되어 있습니다. 에디터도 허브와 마찬가지로 드라이브 바로 다음에 설치하는 편이 정말 좋습니다. 환경 설정 눌러보시면 프로젝트가 있습니다. 저는 c 드라이브 바로 아래에 유니티 플레이트라는 폴더를 만들어서 지정해 놓았습니다. 이렇게 해두면 나중에 플랫폼 빌드할 때 한글 폴더에 의해서 문제가 발생하지 않습니다. 그래서 미리 이렇게 하는 걸 추천드리고, 설치 탭에 가시면 에디터와 컨텐츠 이런 식으로 깔끔하게 정리했습니다. 이렇게 하면 제 c 드라이브가 깔끔하게 정리됩니다. 그래서 에디터를 설치할 거 아닙니까? 저는 이미 설치가 되어 있지만 여러분들은 없을 거예요. 에디터를 설치 버튼을 눌러 주시고 이번 뱀서라이

In [14]:
contents = repository.find_all()
for content in contents[1:]:
    content.set_script(None)
    repository.save(content)